In [4]:
import urllib.request

query = '세탁기'
keys = []

with open('API_KEY.txt', 'r') as key:
    for line in key.readlines():
        text = line.strip()
        if text != '':
            keys.append(text.split(':')[1])

encText = urllib.parse.quote(query)
url = 'https://openapi.naver.com/v1/search/shop?query=' + encText # JSON 결과

request = urllib.request.Request(url)
request.add_header('X-Naver-Client-Id',keys[0])
request.add_header("X-Naver-Client-Secret",keys[1])


response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)






{
	"lastBuildDate":"Fri, 14 Apr 2023 15:17:26 +0900",
	"total":2261989,
	"start":1,
	"display":10,
	"items":[
		{
			"title":"LG전자 통돌이 TR12WL",
			"link":"https:\/\/search.shopping.naver.com\/gate.nhn?id=21225654919",
			"image":"https:\/\/shopping-phinf.pstatic.net\/main_2122565\/21225654919.20220415093405.jpg",
			"lprice":"311670",
			"hprice":"",
			"mallName":"네이버",
			"productId":"21225654919",
			"productType":"1",
			"brand":"통돌이",
			"maker":"LG전자",
			"category1":"디지털\/가전",
			"category2":"생활가전",
			"category3":"세탁기",
			"category4":"일반세탁기"
		},
		{
			"title":"LG전자 트롬 F9WKB",
			"link":"https:\/\/search.shopping.naver.com\/gate.nhn?id=16567044284",
			"image":"https:\/\/shopping-phinf.pstatic.net\/main_1656704\/16567044284.20210526125306.jpg",
			"lprice":"423500",
			"hprice":"",
			"mallName":"네이버",
			"productId":"16567044284",
			"productType":"1",
			"brand":"트롬",
			"maker":"LG전자",
			"category1":"디지털\/가전",
			"category2":"생활가전",
			"category3":"세탁기",
			"category4":"드

In [ ]:
# 네이버 검색 사이트
import webbrowser
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By

# naver_search = 'https://search.naver.com/search.naver?query='
# search_word = '고양이'



params = {
    'query' : '세탁기',
    'sort' : 'timestamp',
}


url = 'https://search.shopping.naver.com/search/all'

data = requests.get(url, params=params)

soup = BeautifulSoup(data.text)
div = soup.select_one('div.list_basis')
a = div.select('a.basicList_etc__LSkN_')

for i in a:
    print(i)
        

In [ ]:
import webbrowser

url = 'https://shopping.naver.com/home'
webbrowser.open(url)

In [ ]:
# 단일 크롤링
import time


driver = webdriver.Chrome()
driver.get("https://www.google.co.kr/imghp?hl=ko&ogbl") # 이미지 사이트를 한 번에 접속
# assert "Python" in driver.title

elem = driver.find_element(By.NAME, 'q')
elem.clear()
elem.send_keys("해리포터") # 검색어를 확인
elem.send_keys(Keys.RETURN) # enter 키를 친다


driver.find_elements(By.CSS_SELECTOR, '.rg_i.Q4LuWd')[0].click() # 큰 그림을 클릭
time.sleep(3)
src = driver.find_element(By.CSS_SELECTOR, '.n3VNCb').get_attribute('src') # src 주소를 가져와야한다.

# 이미지 다운로드
u.request.urlretrieve(src, 'test.png')



# n3VNCb KAlRDb



# https://upload.wikimedia.org/wikipedia/ko/d/dd/%ED%95%B4%EB%A6%AC_%ED%8F%AC%ED%84%B0%EC%99%80_%EB%A7%88%EB%B2%95%EC%82%AC%EC%9D%98_%EB%8F%8C_%EC%98%81%ED%99%94.jpg

# assert "No results found." not in driver.page_source
# driver.close()